In [27]:
import sqlite3
import pandas as pd
from pathlib import Path
db_loc = Path(r'C:\Main\Data\_\Database\sqlite\RespData_2205.db')
csv_loc = Path(r'C:\Users\HY_Burger\Desktop\Project')
def BasicQuery(table_in):
    table_f = 'PatientInfo_Total'
    query_ = '''
        SELECT A.*, B.END_s
        FROM {0} as A
        INNER JOIN {1} as B
        ON A.PID = B.PID
        GROUP by A.PID
    '''.format(table_f, table_in)
    with sqlite3.connect(db_loc) as con:
        df_sql = pd.read_sql(query_, con)
    return df_sql
remark_l = ['remark','RemarkIn','RemarkOut','RemarkInICU','RemarkOutICU']
def NotHeadRemarkPid(df):
    pid_ = []
    for i in df.index:
        row = [df.loc[i, j] for j in remark_l]
        row = [j for j in row if j]
        row = [False if '脑' in j or '神经' in j else True for j in row]
        if False in row:
            pass
        else:
            pid_.append(df.loc[i, 'PID'])
    return pid_
def InfoCollect(df):
    info = 'Total:{0}, Succ:{1}, Fail:{2}'.format(df.shape[0], len(df[df.END_s==0]), len(df[df.END_s==1]))
    return info

In [28]:
def MainProcess(form_name):
    df_0 = BasicQuery(form_name)
    df_0.END_s = [0 if '成功' in i else 1 for i in df_0.END_s]
    # df_1 = pd.read_csv(csv_loc/(form_name + '.csv'))
    print('Raw: \n' + InfoCollect(df_0))
    df_0 = df_0[df_0.age <= 75]
    print('Sub Old: \n' + InfoCollect(df_0))
    pid_select = NotHeadRemarkPid(df_0)
    df_0 = df_0[df_0['PID'].isin(pid_select)]
    print('Sub Head&Nerv: \n' + InfoCollect(df_0))
    # df_1 = df_1[df_1['PID'].isin(list(df_0.pid))]
    # pd.DataFrame.to_csv(df_1, 'prepare.csv', index=False)

In [30]:
process_mode = ['Extube_SumP12', 'Extube_PSV', 'Extube_NotPSV','Wean_SumP12', 'Wean_PSV']
MainProcess('Wean_SumP12')

Raw: 
Total:722, Succ:519, Fail:203
Sub Old: 
Total:534, Succ:403, Fail:131
Sub Head&Nerv: 
Total:269, Succ:218, Fail:51
